In [1]:
import json
from bs4 import Tag
from tqdm import tqdm
import os
from concurrent.futures import ThreadPoolExecutor, as_completed


In [2]:
import requests
from bs4 import BeautifulSoup
import re
import time
import random
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from datetime import datetime, timedelta

def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session


In [3]:
def each_link_content(url):
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-encoding': 'gzip, deflate, br, zstd',
        'accept-language': 'en-US,en;q=0.9,en-IN;q=0.8',
        'cache-control': 'max-age=0',
        'cookie': 'VEE=wikitext; bowikimwclientpreferences=vector-feature-limited-width-clientpref-0; GeoIP=IN:::22.00:79.00:v4; bowikimwuser-sessionId=57993a543a8ba6c65a70; WMF-Last-Access=25-Sep-2024; WMF-Last-Access-Global=25-Sep-2024; bowikiPreventProbe=1; NetworkProbeLimit=0.001',
        'priority': 'u=0, i',
        'referer': 'https://bo.wikipedia.org/wiki/Special:AllPages',
        'sec-ch-ua': '"Microsoft Edge";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36 Edg/129.0.0.0'
    }
    
    
    final_response = {
        "data": {
            'title': "",
            'body': {"Text": []},
            'meta_data': {'URL': url,}
        },
        "Message": "Success",
        "Response": 200
    }
    
    try:
        # Add a random delay before making the request
        time.sleep(random.uniform(1, 3))
        
        # Make the request to the URL using the retry session
        session = requests_retry_session()
        response = session.get(url, headers=headers, allow_redirects=False)
        response.raise_for_status()
        
        # Check for redirect
        if response.is_redirect:
            final_response["Message"] = f"Redirected to: {response.headers['Location']}"
            final_response["Response"] = response.status_code
            return final_response
        
        # Parse the page content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        # Remove script and style elements
        for script in soup(["script", "style"]):
            script.decompose()

        # Extract title
        title = soup.find('span', class_="mw-page-title-main")
        if title:
            title_text = title.get_text(strip=True)
        else:
            title_text = ""
        final_response['data']["title"] = title_text
        
        
        full_body = soup.find('div', id="bodyContent")
        # print(full_body)
        if full_body:
            # Extract body content
            try:
                body = full_body.find("div", class_="mw-content-ltr mw-parser-output")
                # print(body)
                if body:
                    text = body.get_text(separator='\n', strip=True)
                    lines = [line for line in text.split('\n') if line.strip()]
                    # print(lines)
                    if lines:
                        # Extracting all <p> tags for text content
                        final_response['data']['body']["Text"] = lines
                    else:
                        final_response['data']['body']["Text"] = [""]
    
            except AttributeError as e:
                final_response['data']['body']["Text"] = [f"Error fetching body content{str(e)}"]

       
        return final_response
    except requests.Timeout:
        final_response["Message"] = "Request timed out"
        final_response["Response"] = 408  # Request Timeout
        return final_response
    except requests.RequestException as e:
        final_response["Message"] = f"An error occurred while fetching the article: {str(e)}"
        final_response["Response"] = getattr(e.response, 'status_code', 500)
        return final_response
    except Exception as e:
        final_response["Message"] = f"An unexpected error occurred: {e}"
        final_response["Response"] = 500
        return final_response

In [4]:
url = "https://bo.wikipedia.org/wiki/%E0%BD%A8%E0%BC%8B%E0%BD%A3%E0%BC%8B%E0%BD%94%E0%BC%8B%E0%BD%98%E0%BC%8D"
url = "https://bo.wikipedia.org/wiki/%E0%BD%9A%E0%BC%8B%E0%BD%A2%E0%BD%BC%E0%BD%84%E0%BC%8B%E0%BD%9F%E0%BE%B3%E0%BC%8B%E0%BD%96%E0%BD%9F%E0%BD%84%E0%BC%8B%E0%BD%91%E0%BD%82%E0%BE%B2%E0%BC%8B%E0%BD%A0%E0%BD%91%E0%BD%B4%E0%BD%A3%E0%BC%8B"
each_link_content(url)


{'data': {'title': 'ཚ་རོང་ཟླ་བཟང་དགྲ་འདུལ་',
  'body': {'Text': ['ཚ་རོང་ཟླ་བཟང་དགྲ་འདུལ',
    'ཚ་རོང་ཟླ་བཟང་དགྲ་འདུལ་ནི་འཕན་པོ་དཔལ་གླིང་ཤང་ཁ་སྐོར་གྲོང་ཚོའི་མདའ་བཟོ་ཚང་ཞེས་སེ་ར་ཕུར་ལྕོག་བླ་བྲང་གི་མི་སེར་ཞིག་གི་ཁྱིམ་ནས་ཡིན་ཞིང་། ཁྱིམ་ཚང་ཁྲལ་བུན་ཆེ་བས་ཁོང་དང་གཅུང་པོ་བསོད་ནམས་དབང་འདུས་གཉིས་ལྷ་སའི་བྱང་ཕྱོགས་དོག་བདེ་ལྷོ་མོས་གཞིས(དེ་སྔ་ཕུར་ལྕོག་བླ་བྲང་གི་གཞིས་ཀ་ཡིན)སུ་ལུག་རྫི་ར་སྡོད་སྐབས་སྤྱང་ཀིས་ར་ལུག་བསད་རྐྱེན་བྲོས་བྱོལ་གྱིས་ལྷ་སའི་ནུབ་ཏུ་ཡོད་པའི་མཁར་ནག་གླིང་ཁར་བོད་ས་གནས་སྲིད་གཞུང་གི་རྔ་མོང་བདག་སྐྱོང་འགོ་པ་ཞིག་གི་སར་བསྡད་པ་བརྒྱུད་དེའི་ངོ་ཤེས་རྩེ་དྲུང་རྩེ་གཉེར་ཚང་བ་ཞིག་གི་གཡོག་པོར་སྡོད་སྐབས་བོད་རབ་བྱུང་བཅོ་ལྔ་པའི་ཤིང་འབྲུག(༡༩༠༤)ལོར་དབྱིན་ཇིས་བོད་དུ་བཙན་བཛུལ་བྱེད་དུས་ཏཱ་ལའི་བླ་མ་སྐུ་ཕྲེང་བཅུ་གསུམ་པ་རྒྱ་སོག་ཡུལ་གྲུར་ཕེབས་སྐབས་རྩེ་དྲུང་དེ་ཆིབས་ཞབས་སུ་བསྐྱོད་ནས་ཟླ་བཟང་དགྲ་འདུལ་གཡོག་ཏུ་ཡོད་པ་དེའི་གྲུང་ཤ་དོད་པ་སོགས་ཀྱི་རྣམ་འགྱུར་ལ་ཏཱ་ལའི་བླ་མའི་མཉེས་པོ་བྱུང་ནས་ཆིབས་ཞབས་སུ་ཚུད། ལྕགས་ཁྱི་(༡༩༡༠)ལོར་ཏཱ་ལའི་བླ་མ་རྒྱ་གར་དུ་ཕེབས་པའི་ཆིབས་ཞབས་སུ་བསྐྱོད་དེ་ལྕགས་ཟམ་གྲུ་ཁར་ལུའུ་ཅུན་དམག་གི་རྗེས་འདེད་ལ་འགོག་རྒ

In [5]:


# def read_json(path, file_name):
#     try:
#         with open(path+file_name, 'r', encoding='utf-8') as openfile:
#             Loaded_file = json.load(openfile)
#             print(f"Successfully loaded: {file_name}")
#         return Loaded_file
#     except Exception as e:
#         print(f"Error loading {file_name}: {str(e)}")
#         return None

# def get_content(All_links_data, Total_lenght, page_key_code, page_key_list):
#     all_article = {}
    
#     with ThreadPoolExecutor(max_workers=10) as executor:
#         future_to_url = {}
#         start = 1
#         for i in range(start, Total_lenght+1):
#             page_key = page_key_code + str(i)
#             all_link_page = All_links_data["Links"]
            
#             for url in all_link_page:
#                 future = executor.submit(scrape_article, url, page_key_code)
#                 future_to_url[future] = (page_key, url)
        
#         for future in tqdm(as_completed(future_to_url), total=len(future_to_url)):
#             page_key, url = future_to_url[future]
#             try:
#                 article_content = future.result()
#                 if article_content:
#                     article_key = f"{page_key}_scrape_{file_name_code}_Article_{len([k for k in all_article if k.startswith(page_key)]) + 1}"
#                     all_article[article_key] = article_content
#             except Exception as e:
#                 print(f"Error processing {url}: {str(e)}")
    
#     Failure_count = sum(1 for article in all_article.values() if article["Response"] != 200)
#     print(f"Total Failure in the {page_key_list[1]} article: {Failure_count}")
    
#     save_file_name = f"scrape_{page_key_code}_ALL_content.json"
#     print(save_file_name)
#     path = "./data/parallel_content/"
#     save_json(path, save_file_name, all_article)







In [9]:

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Tag):
            return obj.get_text()
        return str(obj)


def save_json(path, file_name, data):
    try: 
        with open(path + file_name, "w", encoding='utf-8') as outfile:
            json.dump(data, outfile, indent=4, ensure_ascii=False, cls=CustomJSONEncoder)
        print(f"Successfully saved: {file_name}")
    except Exception as e:
        print(f"Error saving {file_name}: {str(e)}")


def scrape_article(url):
    try:
        article_content = each_link_content(url)
        return article_content
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None


def get_content(All_links_data, Total_lenght, page_key_code):
    """
    
    """
    try:
        all_article = {}
        all_link_page = All_links_data["Links"]
        for i in tqdm(range(Total_lenght)):
            each_url = all_link_page[i]
            result = scrape_article(each_url)
            article_code = f"{page_key_code}_{i}"
            all_article[article_code] = result

        return all_article


    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {page_key_code}: {str(e)}")
    except Exception as e:
        print(f"Error processing url of file {page_key_code}: {str(e)}")




def process_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            File_name = os.path.basename(file_path)
            All_links_data = json.load(file)
            
            Total_lenght = len(All_links_data["Links"])
            print(f"Total articles in {File_name}: {Total_lenght}")
        
            page_key_list = list(File_name.split("."))
            page_key_code = page_key_list[0]+"_Article"
            print(f"Page key code: {page_key_code}")
            
            processed_data = get_content(All_links_data, Total_lenght, page_key_code)
            save_file_name = f"scrape_{page_key_list[0]}_ALL_content.json"
            if processed_data:
                print(save_file_name)
                path = "./data/parallel_content/"
                save_json(path, save_file_name, processed_data)
            else:
                print(f"failed to save the {save_file_name}")
            
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {os.path.basename(file_path)}: {str(e)}")
    except Exception as e:
        print(f"Error processing file {os.path.basename(file_path)}: {str(e)}")


In [ ]:

directory = "./data/links/"
json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
print(f"Total Files {len(json_files)}")

# json_files = json_files[:5]
with ThreadPoolExecutor(max_workers=10) as executor:
    list(tqdm(executor.map(process_json_file, json_files), total=len(json_files)))

print(f"Processed {len(json_files)} files")


Total Files 43
Total articles in Page 2.json: 345
Page key code: Page 2_Article
Total articles in Page 1.json: 345
Page key code: Page 1_Article
Total articles in Page 4.json: 345
Page key code: Page 4_Article


  0%|          | 0/345 [00:00<?, ?it/s]

Total articles in Page 6.json: 345
Page key code: Page 6_Article
Total articles in Page 3.json: 345
Page key code: Page 3_Article
Total articles in Page 5.json: 345
Page key code: Page 5_Article
Total articles in Page 7.json: 345
Page key code: Page 7_Article
Total articles in Page 8.json: 345
Page key code: Page 8_Article
Total articles in Page 9.json: 345
Page key code: Page 9_Article
Total articles in Page 10.json: 345
Page key code: Page 10_Article




  0%|          | 0/345 [00:00<?, ?it/s]



  0%|          | 0/345 [00:00<?, ?it/s]




  0%|          | 0/345 [00:00<?, ?it/s]


  0%|          | 0/345 [00:00<?, ?it/s]





  0%|          | 0/345 [00:00<?, ?it/s]








  0%|          | 0/345 [00:00<?, ?it/s]









  0%|          | 0/345 [00:00<?, ?it/s]







  0%|          | 0/345 [00:00<?, ?it/s]






  0%|          | 0/43 [00:00<?, ?it/s]





  0%|          | 1/345 [00:06<38:04,  6.64s/it]









  0%|          | 1/345 [00:07<42:54,  7.48s/it]



  0%|          | 1/345 [00:07<43:45,  7.63s/it]

  0%|          | 1/345 [00:07<43:58,  7.67s/it]




  0%|          | 1/345 [00:07<45:43,  7.97s/it]








  0%|          | 1/345 [00:08<45:54,  8.01s/it]


  0%|          | 1/345 [00:08<46:21,  8.08s/it]







  0%|          | 1/345 [00:09<51:47,  9.03s/it]



  1%|          | 2/345 [00:11<31:58,  5.59s/it]

  1%|          | 2/345 [00:12<32:00,  5.60s/it]





  1%|          | 2/345 [00:12<33:02,  5.78s/it]








  1%|      

scrape_Page 3_ALL_content.json
Successfully saved: scrape_Page 3_ALL_content.json
Total articles in Page 11.json: 345
Page key code: Page 11_Article


...






 94%|█████████▎| 323/345 [40:36<02:47,  7.63s/it]









100%|█████████▉| 344/345 [39:52<00:07,  7.05s/it]



  5%|▌         | 18/345 [02:11<39:45,  7.30s/it]

  3%|▎         | 12/345 [01:31<40:31,  7.30s/it]




 94%|█████████▍| 324/345 [40:44<02:39,  7.57s/it]









100%|██████████| 345/345 [40:00<00:00,  6.96s/it]


scrape_Page 37_ALL_content.json
Successfully saved: scrape_Page 37_ALL_content.json


  6%|▌         | 19/345 [02:18<39:43,  7.31s/it]



  6%|▋         | 22/345 [02:40<39:52,  7.41s/it]

  4%|▍         | 13/345 [01:40<42:17,  7.64s/it]




  6%|▌         | 20/345 [02:26<40:18,  7.44s/it]]



  7%|▋         | 23/345 [02:48<40:33,  7.56s/it]

  4%|▍         | 14/345 [01:46<40:31,  7.34s/it]




 94%|█████████▍| 326/345 [40:59<02:26,  7.70s/it]



  6%|▌         | 21/345 [02:33<40:34,  7.51s/it]

  4%|▍         | 15/345 [01:54<41:41,  7.58s/it]



  7%|▋         | 25/345 [03:01<38:11,  7.16s/it]




  6%|▋         | 22/345 [02:41<41:15,  7.66s/it]]

  5%|▍         | 16/345 [02:02<41:58,  7.66s/it]



  8%|▊         | 26/345 [03:09<38:45,  7.29s/it]




  7%|▋         | 23/345 [02:49<41:34,  7.75s/it]]

  5%|▍         | 17/345 [02:09<40:56,  7.49s/it]




 95%|█████████▌| 329/345 [41:22<02:00,  7.53s/it]



  7%|▋         | 24/345 [02:56<40:04,  7.49s/it]

  5%|▌         | 18/345 [02:17<41:19,  7.58s/it]




 96%|█████████▌| 330/345 [41:30<01:54,  7.64s/it]



  7%|▋      

scrape_Page 33_ALL_content.json
Successfully saved: scrape_Page 33_ALL_content.json






 12%|█▏        | 43/345 [04:58<32:03,  6.37s/it]

 11%|█         | 37/345 [04:17<33:56,  6.61s/it]



 13%|█▎        | 44/345 [05:02<29:00,  5.78s/it]

 13%|█▎        | 45/345 [05:10<31:45,  6.35s/it]



 14%|█▍        | 48/345 [05:32<32:32,  6.57s/it]

 13%|█▎        | 46/345 [05:18<34:06,  6.84s/it]



 14%|█▍        | 49/345 [05:40<34:12,  6.93s/it]

 12%|█▏        | 40/345 [04:37<33:18,  6.55s/it]



 14%|█▍        | 50/345 [05:45<31:01,  6.31s/it]

 14%|█▍        | 48/345 [05:30<32:28,  6.56s/it]

 12%|█▏        | 42/345 [04:48<31:33,  6.25s/it]



 15%|█▍        | 51/345 [05:52<32:33,  6.64s/it]



 14%|█▍        | 49/345 [05:35<29:56,  6.07s/it]

 12%|█▏        | 43/345 [04:53<29:28,  5.86s/it]



 14%|█▍        | 50/345 [05:43<32:15,  6.56s/it]

 13%|█▎        | 44/345 [05:02<33:11,  6.62s/it]



 15%|█▍        | 51/345 [05:47<29:00,  5.92s/it]

 13%|█▎        | 45/345 [05:05<28:45,  5.75s/it]



 16%|█▌        | 55/345 [06:15<29:41,  6.14s/it]

 15%|█▌        | 52/345 [05: